In [ ]:
## Experimental Setup

In [1]:
from fastai.vision import *
from fastai.distributed import *
from fastai.metrics import error_rate
from fastai.callbacks import SaveModelCallback

if torch.cuda.is_available():
  print("Great! Good to go!")
else:
  print('CUDA is not up!')

Great! Good to go!


In [2]:
# from gepcore.utils import cell_graph
# from gepnet.model_v2 import get_gepnet, arch_config
# from gepnet.utils import count_parameters
# torch.backends.cudnn.benchmark = True

from gepcore.utils import cell_graph, convolution
from gepcore.entity import Gene, Chromosome
from gepcore.symbol import PrimitiveSet
from gepnet.model_v1 import get_net, arch_config
from gepnet.utils import count_parameters
torch.backends.cudnn.benchmark = True

In [3]:
from pygraphviz import AGraph
import glob

In [4]:
#comp_graphs = []
# for i in range(3):
#     graph = [AGraph(g) for g in glob.glob('comp_graphs/new/{}/*.dot'.format(i))]
#     _, comp_graph = cell_graph.generate_comp_graph(graph)#
#     comp_graphs.append(comp_graph)
#     #print(comp_graph, '\n')

graph = [AGraph(g) for g in glob.glob('comp_graphs/new/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)#

In [4]:
# generate new chromosome
# define primitive set
pset = PrimitiveSet('cnn')

# add cellular encoding program symbols
pset.add_program_symbol(cell_graph.end)
pset.add_program_symbol(cell_graph.seq)
pset.add_program_symbol(cell_graph.cpo)
pset.add_program_symbol(cell_graph.cpi)

# add convolutional operations symbols
conv_symbol = convolution.get_symbol()
pset.add_cell_symbol(conv_symbol.conv3x3)
#pset.add_cell_symbol(conv_symbol.sepconv3x3)
#pset.add_cell_symbol(conv_symbol.sepconv5x5)
#pset.add_cell_symbol(conv_symbol.dilconv3x3)
#pset.add_cell_symbol(conv_symbol.dilconv5x5)
pset.add_cell_symbol(conv_symbol.conv1x1)
#pset.add_cell_symbol(conv_symbol.dwconv3x3)
#pset.add_cell_symbol(conv_symbol.maxpool3x3)

# comp_graphs = []
# for i in range(3):
#     def gene_gen():
#         return Gene(pset, 2)
#     ch = Chromosome(gene_gen, 3)
#     graph, comp_graph = cell_graph.generate_comp_graph(ch)
#     comp_graphs.append(comp_graph)
#     cell_graph.save_graph(graph, 'comp_graphs/new/{}'.format(i))
#     cell_graph.draw_graph(graph, 'comp_graphs/new/{}'.format(i))


def gene_gen():
    return Gene(pset, 2)
ch = Chromosome(gene_gen, 4)
graph, comp_graph = cell_graph.generate_comp_graph(ch)
cell_graph.save_graph(graph, 'comp_graphs/new/')
cell_graph.draw_graph(graph, 'comp_graphs/new/')

In [5]:
# import random
seed = 200
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed(http://localhost:8888/notebooks/ResearchProjects/geppy_nn/evolution_search.ipynb#seed)
# torch.cuda.manual_seed_all(seed)

# # enable torch backends
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
#torch.backends.deterministic = True

In [6]:
conf = arch_config(comp_graph=comp_graph,
                   #depth_coeff=1.0,
                   #width_coeff=1.0,
                   channels=40,
                   repeat_list=[4, 4, 4],
                   classes=10)

net = get_net(conf)
count_parameters(net)

7.30149

In [7]:
size=32
cutout_frac = 0.25
p_cutout = 0.75
cutout_sz = round(size*cutout_frac)
cutout_tfm = cutout(n_holes=(2,2), length=(cutout_sz, cutout_sz), p=p_cutout)

path = untar_data(URLs.CIFAR)
tfms = get_transforms()#xtra_tfms=[cutout_tfm])#do_flip=False,
                      #xtra_tfms=[RandTransform(tfm=cutout, kwargs={'n_holes': (1,1)})])
bs = 128
data = (ImageList.from_folder(path/'train')
        .split_by_rand_pct(valid_pct=0.1, seed=seed) 
        .label_from_folder()
        .transform(tfms, size=32)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize(cifar_stats))

In [8]:
#opt_func=optim.Adam,
model_dir = '/home/cliff/ResearchProjects/geppy_nn/comp_graphs/new'
learn = Learner(data, net, bn_wd=False, metrics=[error_rate, accuracy], model_dir=model_dir).mixup(alpha=1.) 
learn.to_fp16()   

Learner(data=ImageDataBunch;

Train: LabelList (45000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
automobile,automobile,automobile,automobile,automobile
Path: /home/cliff/.fastai/data/cifar10/train;

Valid: LabelList (5000 items)
x: ImageList
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
y: CategoryList
cat,cat,ship,ship,automobile
Path: /home/cliff/.fastai/data/cifar10/train;

Test: None, model=Network(
  (stem_): Sequential(
    (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blocks_): Sequential(
    (0): Cell(
      (relu): ReLU(inplace=True)
      (branch_0): Layer(
        (conv3x3_0): Sequential(
          (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(40, eps=1e-05

In [ ]:
learn.lr_find(wd=4e-4,end_lr=100)
learn.recorder.plot()
#SGD(params, lr=<required parameter>, momentum=0, dampening=0, weight_decay=0, nesterov=False)

In [ ]:
cb = SaveModelCallback(learn, every='improvement', monitor='accuracy', name='model_0')
learn.fit_one_cycle(1500, 1e-2, wd=0.0004, callbacks=[cb]) 

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,1.747732,1.432521,0.502600,0.497400,02:40
1,1.570998,1.158666,0.400000,0.600000,01:39
2,1.467275,0.912237,0.307000,0.693000,01:41
3,1.401526,0.843324,0.277600,0.722400,01:41
4,1.353326,0.835318,0.262800,0.737200,01:41
5,1.317201,0.690659,0.216600,0.783400,01:40
6,1.276403,0.704671,0.228400,0.771600,01:41
7,1.254399,0.640482,0.202800,0.797200,01:40
8,1.245871,0.608457,0.190000,0.810000,01:41
9,1.214579,0.595780,0.179800,0.820200,01:41


Better model found at epoch 0 with accuracy value: 0.4973999857902527.
Better model found at epoch 1 with accuracy value: 0.6000000238418579.
Better model found at epoch 2 with accuracy value: 0.6930000185966492.
Better model found at epoch 3 with accuracy value: 0.7224000096321106.
Better model found at epoch 4 with accuracy value: 0.7372000217437744.
Better model found at epoch 5 with accuracy value: 0.7833999991416931.
Better model found at epoch 7 with accuracy value: 0.7972000241279602.
Better model found at epoch 8 with accuracy value: 0.8100000023841858.
Better model found at epoch 9 with accuracy value: 0.8202000260353088.
Better model found at epoch 10 with accuracy value: 0.84579998254776.
Better model found at epoch 11 with accuracy value: 0.8574000000953674.
Better model found at epoch 14 with accuracy value: 0.8740000128746033.
Better model found at epoch 16 with accuracy value: 0.8903999924659729.
Better model found at epoch 20 with accuracy value: 0.8944000005722046.
Bet

In [ ]:
####################### Testing ###########################

In [ ]:
# graph_dir = 'comp_graphs/new/*.dot' #'/content/mnt/My Drive/Colab Notebooks/ResearchProject/comp_graphs/experiment_1/best/indv_1/*.dot'
# graph = [AGraph(g) for g in glob.glob(graph_dir)]
# _, comp_graph = cell_graph.generate_comp_graph(graph)

# comp_graphs = []
# for i in range(3):
#     graph = [AGraph(g) for g in glob.glob('comp_graphs/new/{}/*.dot'.format(i))]
#     _, comp_graph = cell_graph.generate_comp_graph(graph)#
#     comp_graphs.append(comp_graph)

graph = [AGraph(g) for g in glob.glob('comp_graphs/new/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)#

In [ ]:
conf = arch_config(comp_graphs=comp_graphs,
                   #depth_coeff=1.0,
                   #width_coeff=1.0,
                   channels=40,
                   repeat_list=[3, 2, 3],
                   classes=10)

net = get_net(conf)
count_parameters(net)

In [ ]:
tfms = get_transforms(do_flip=False)
path = untar_data(URLs.CIFAR)
bs = 256

data = (ImageList.from_folder(path)
        .split_by_folder(train='train', valid='test')
        .label_from_folder()
        .transform(tfms, size=32)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize(cifar_stats))

In [ ]:
model_dir = '/home/cliff/ResearchProjects/geppy_nn/comp_graphs/new' #'/content/mnt/My Drive/Colab Notebooks/ResearchProject/models/'
model = Learner(data, net, metrics=[accuracy, error_rate]).load(model_dir+'/model_1')
model.validate()

In [ ]:
preds,y,losses = model.get_preds(ds_type=DatasetType.Valid, with_loss=True)
preds[:5]